In [1]:
import pandas as pd

# Fixing data labels

In working with pandas dataframes, you will come across datasets that looks like this.

In [2]:
df1 = pd.read_csv("./cmc.csv")
df1[1:5]

,24,2,3,3.1,1,1.1,2.1,3.2,0,1.2
1,43,2,3,7,1,1,3,4,0,1
2,42,3,2,9,1,1,3,3,0,1
3,36,3,3,8,1,1,3,2,0,1
4,19,4,4,0,1,1,3,3,0,1


This is a problem because you don't see what the data actually is representing. You could go back and open CSV file to fix this, but in pandas there is a easy way of adjusting column.

In [3]:
column_names = ["age", 
             "education", 
             "higher_education", 
             "num_children", 
             "practices_islam", 
             "working", 
             "occupation",
             "solo_index",  # standard of living index
             "media_exposure",
             "contraceptive_method"
            ]
df2 = pd.read_csv("./cmc.csv", header=None, names=column_names, encoding='latin-1')
df2[1:5]

,age,education,higher_education,num_children,practices_islam,working,occupation,solo_index,media_exposure,contraceptive_method
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1
3,42,3,2,9,1,1,3,3,0,1
4,36,3,3,8,1,1,3,2,0,1


Now, this is a dataset we can work with.

# Crosstab

Sometime when you want to investigate the association between two variables in a dataset, you will construct what statisticians call contingency table. Pandas implements "Crosstab" functionality which stands for cross tabulation to construct this. Having the regiment data we can play with it.

In [4]:
# https://chrisalbon.com/python/data_wrangling/pandas_crosstabs/
pd.crosstab(rdf.regiment, rdf.company, margins=True)

NameError: name 'rdf' is not defined

This shows the frequency of regiments in different companies. You can see that that each company has 2 Dragoons, 2 Nighthawks, and 2 Scouts.

# Duplicates

Duplicates may be a easy problem to deal with, but by no means is it negligible in impact it has in assessing data. Removing duplicates, along with dealing with missing data is one of the most fundamental things we need to do to assure the quality of our data. Thanks to Pandas, we have eay ways of detecting and dealing with the duplicates.

In [ ]:
ddf = pd.DataFrame(
    [
        ["Coyle", "Cool"],
        ["Jake", "Handsome"],
        ["Lee", "Smart"],
        ["Coyle", "Cool"]
    ], 
    columns={
        "Name", 
        "Defining Trait"
    }
)

ddf

We can see that the last row is actually the repeat of the first row

In [ ]:
ddf.duplicated()

`duplicated()` shows if one row is duplicate of the other with boolean. In this case, to assure the quality of data we need to remove the last row.

In [ ]:
ddf.drop_duplicates()

That easy :)

# Merge

Condier the following dataframes

In [ ]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""
# Styling Side By Side
HTML('<style>{}</style>'.format(CSS))

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/merging.html
left = pd.DataFrame({
         'k': ['K0', 'K1', 'K1', 'K2'],
         'lv': [1, 2, 3, 4],
         's': ['a', 'b', 'c', 'd']
        })

right = pd.DataFrame({
    'k': ['K1', 'K2', 'K4'],
    'rv': [1, 2, 3]
})

In [ ]:
display(left)
display(right)

Here we have two different dataframes with same column names, presumably two table representing different parts of the same set. To complete the data, we can merge the two tables and work with only complete set of values!

In [ ]:
merged = pd.merge(left, right, how="inner", sort=True, copy=True)
merged

Or, if we want to create a dataframe of all possible values even those including NaN, we can do outer join.

In [ ]:
merged_outer = pd.merge(left, right, how="outer", sort=True, copy=True)
merged_outer

Now what do we do with the NaN values? I will tell you in the next section.

# Dealing with missing values

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/missing_data.html
mdf = pd.DataFrame(
        np.random.randn(5, 3), 
        index=['a', 'c', 'e', 'f', 'h'],
        columns=['one', 'two', 'three']
    )
mdf['four'] = 'bar'
mdf['five'] = mdf['one'] > 0
mdf = mdf.reindex(
        ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    )

mdf

Here we have NaNs in the places where we should have booleans and strings. We have already seen `dropna()` function, however dropping missing data is rairly the answer. We still need to compute the data and less the data lower the credibility of the result. One dimple way of filling out this data is by using `fillna()` function.

In [ ]:
mdf.fillna(mdf.mean())

You can see the first three columns will have value that makes sense, but the last two does not. Filling the missing values with mean of our current value is a common and effective way of dealing with missing data, but it doesn't always work. Notice the column 5 which is supposed to have boolean has 0.6 as the value that comes from `mdf().mean()`


In [ ]:
print(True == 1)
print(False == 0)
print(((3*1)+(2*0))/5)
print(mdf['five']['b'])

You can see that for a discrete binary value pandas will try to compute a "average" that should not be there. Notice how the value 0.6 appears as nan. We cannot work with this value. So, we can tweak this a little.

In [ ]:
mdf['one'] = mdf.fillna(mdf.mean())
mdf['two'] = mdf.fillna(mdf.mean())
mdf['three'] = mdf.fillna(mdf.mean())

In [ ]:
mdf['four'] = 'bar'
mdf['five'] = mdf['one'] > 0
# print(mdf)
mdf

If a value is part of the input and was provided as missing, the only way to fill in this value is by finding the average or replacing it with any value of one's choice. However for values of column 5 that we derived from column 1. In this case, the best way to replace the missing data is by recomputing it.